In [1]:
import tensorflow as tf

In [2]:
!pip install rdkit-pypi

In [3]:
!pip install fcd

In [4]:
import os

import numpy as np
import pandas as pd
import pickle
from fcd import get_fcd, load_ref_model,canonical_smiles, get_predictions, calculate_frechet_distance

np.random.seed(0)

In [5]:
model = load_ref_model()

In [6]:
def print_fcd(gen_mol_file, mu1, sigma1):
  global model
  gen_mol = pd.read_csv(gen_mol_file,header=None)[0]

  sample2 = np.random.choice(gen_mol, 10000, replace=False)
  can_sample2 = [w for w in canonical_smiles(sample2) if w is not None]

  act2 = get_predictions(model, can_sample2)

  mu2 = np.mean(act2, axis=0)
  sigma2 = np.cov(act2.T)
  fcd_score = calculate_frechet_distance(
    mu1=mu1,
    mu2=mu2, 
    sigma1=sigma1,
    sigma2=sigma2)
  return fcd_score

In [7]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('.') if isfile(join('.', f))]
FCD_values = {}
gen_mol = pd.read_csv('chembl_22_clean_1576904_sorted_std_final.smi', header=None)[0]
for j in range(5):
  np.random.seed(j)
  sample1 = np.random.choice(gen_mol, 10000, replace=False)
  can_sample1 = [w for w in canonical_smiles(sample1) if w is not None]
  act1 = get_predictions(model, can_sample1)
  mu1 = np.mean(act1, axis=0)
  sigma1 = np.cov(act1.T)
  for i in onlyfiles:
    if '30k' in i:
      print(i, j)
      if j == 0:
        FCD_values[i] = 0
      FCD_values[i] = FCD_values[i] + print_fcd(i, mu1, sigma1)

for i in onlyfiles:
  if '30k' in i:
    FCD_values[i] = FCD_values[i]/5

print(FCD_values)

stack_30k_logp_max.smi 0
dnc_30k_logp_min.smi 0
baseline_lstm_30k_unbiased.smi 0
stack_30k_unbiased.smi 0
ntm_30k_benzene_min.smi 0
ntm_30k_benzene_max.smi 0
stack_30k_benzene_max.smi 0
baseline_gru_30k_unbiased.smi 0
baseline_simple_30k_unbiased.smi 0
ntm_30k_unbiased.smi 0
dnc_30k_benzene_max.smi 0
dnc_30k_unbiased.smi 0
ntm_30k_logp_min.smi 0
dnc_30k_logp_max.smi 0
ntm_30k_logp_max.smi 0
dnc_30k_benzene_min.smi 0
stack_30k_logp_min.smi 0
stack_30k_benzene_min.smi 0
stack_30k_logp_max.smi 1
dnc_30k_logp_min.smi 1
baseline_lstm_30k_unbiased.smi 1
stack_30k_unbiased.smi 1
ntm_30k_benzene_min.smi 1
ntm_30k_benzene_max.smi 1
stack_30k_benzene_max.smi 1
baseline_gru_30k_unbiased.smi 1
baseline_simple_30k_unbiased.smi 1
ntm_30k_unbiased.smi 1
dnc_30k_benzene_max.smi 1
dnc_30k_unbiased.smi 1
ntm_30k_logp_min.smi 1
dnc_30k_logp_max.smi 1
ntm_30k_logp_max.smi 1
dnc_30k_benzene_min.smi 1
stack_30k_logp_min.smi 1
stack_30k_benzene_min.smi 1
stack_30k_logp_max.smi 2
dnc_30k_logp_min.smi 2
baseli

In [9]:
import json
print(json.dumps(FCD_values, sort_keys=True, indent = 4))

{
    "baseline_gru_30k_unbiased.smi": 1.226416423488135,
    "baseline_lstm_30k_unbiased.smi": 0.5912900174112139,
    "baseline_simple_30k_unbiased.smi": 4.998389970929599,
    "dnc_30k_benzene_max.smi": 22.534239579154367,
    "dnc_30k_benzene_min.smi": 11.736028111123073,
    "dnc_30k_logp_max.smi": 17.892360038129894,
    "dnc_30k_logp_min.smi": 6.9168325929467045,
    "dnc_30k_unbiased.smi": 0.7056280910444457,
    "ntm_30k_benzene_max.smi": 17.226944577587055,
    "ntm_30k_benzene_min.smi": 25.988842335690663,
    "ntm_30k_logp_max.smi": 18.13746065742304,
    "ntm_30k_logp_min.smi": 8.121069622491694,
    "ntm_30k_unbiased.smi": 0.6003370726663831,
    "stack_30k_benzene_max.smi": 15.133152970601648,
    "stack_30k_benzene_min.smi": 15.188526713143734,
    "stack_30k_logp_max.smi": 8.804444928305239,
    "stack_30k_logp_min.smi": 10.766788660591887,
    "stack_30k_unbiased.smi": 0.5460475433498942
}
